In [3]:
import mne
import re
from pathlib import Path
import matplotlib
matplotlib.use('TkAgg')
import numpy as np

file_path = Path("./files/S100/S100R01.edf")
raw = mne.io.read_raw_edf(str(file_path), preload=True)



Extracting EDF parameters from C:\Users\Hp\OneDrive\Desktop\All Bout C\Projects\eeg-dataset\files\S100\S100R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...


In [4]:
def clean_names(name):
    return re.sub(r'\W+' , '', name).upper()

raw.rename_channels({ch:clean_names(ch) for ch in raw.ch_names})

montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage, match_case = False)
raw.compute_psd().plot(amplitude = False)

Effective window size : 12.800 (s)
Plotting power spectral density (dB=True).


<MNELineFigure size 1000x350 with 2 Axes>

Channels marked as bad:
none


In [5]:
raw.plot(duration = 10, n_channels = 10, scalings = {'eeg':100e-6})

Using matplotlib as 2D backend.


<MNEBrowseFigure size 800x800 with 4 Axes>

In [9]:
raw.compute_psd(fmax = 12, fmin=8).plot()

Effective window size : 12.800 (s)
Plotting power spectral density (dB=True).


<MNELineFigure size 1000x350 with 2 Axes>

In [6]:
raw.filter(l_freq = 1, h_freq= None)
picks = mne.pick_types(raw.info, eeg=True, meg=True, eog=True, exclude=[])

ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw, picks=picks)
ica.plot_components()


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 529 samples (3.306 s)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 2.6s.


<MNEFigure size 958x981 with 20 Axes>

In [8]:
ica.exclude = [0,5,10,15]

orig_raw = raw.copy()
ica.apply(raw)
raw.plot(duration = 10, n_channels = 10,  scalings = {'eeg':100e-6}, title = 'ica_preprocessed')
orig_raw.plot(duration = 10, n_channels = 10,  scalings = {'eeg':100e-6}, title='original')

Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 4 ICA components
    Projecting back using 64 PCA components


<MNEBrowseFigure size 800x800 with 4 Axes>

In [7]:
raw.compute_psd().plot(amplitude = False)

Effective window size : 12.800 (s)
Plotting power spectral density (dB=True).


<MNELineFigure size 1000x350 with 2 Axes>

In [8]:
raw.compute_psd(fmax = 12, fmin=8).plot()

Effective window size : 12.800 (s)
Plotting power spectral density (dB=True).


<MNELineFigure size 1000x350 with 2 Axes>

In [29]:
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: [np.str_('T0')]


In [45]:
epochs = mne.Epochs(raw, events, event_id, tmin=0, tmax=0.8,baseline=(0, 0),preload = True)

Not setting metadata
1 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 129 original time points ...
0 bad epochs dropped


In [46]:
print(epochs)

<Epochs | 1 events (all good), 0 – 0.8 s (baseline 0 – 0 s), ~117 KiB, data loaded,
 np.str_('T0'): 1>


In [47]:
epochs.plot()

<MNEBrowseFigure size 800x800 with 4 Axes>

In [58]:
epochs.filter(l_freq=1.0, h_freq=40.0).plot(n_channels=20, block = True)

Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)



C:\Users\Hp\AppData\Local\Temp\ipykernel_15504\1452804405.py:1: RuntimeWarning: filter_length (529) is longer than the signal (129), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(l_freq=1.0, h_freq=40.0).plot(n_channels=20, block = True)


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


<MNEBrowseFigure size 1920x991 with 4 Axes>

In [63]:
print(epochs.events[: ,2])

[1]
